In [1]:
# Defining the required access keys for uploading the experiment with the server
%env AWS_ACCESS_KEY_ID=XSHDLTACETFLQURZSAUJ
%env AWS_SECRET_ACCESS_KEY=ISvyN9Aay40ZIaMuQcngOCJdzkSqR85ON1ng9PNZ
%env MLFLOW_S3_ENDPOINT_URL=http://s3.padre-lab.eu
%env MLFLOW_TRACKING_URI=http://mlflow.padre-lab.eu

## Initialize the tracking system of PyPads

In [2]:
from pypads.app.base import PyPads

tracker = PyPads(uri="http://mlflow.padre-lab.eu")
tracker.activate_tracking()
tracker.start_track(experiment_name="PyTorch Example")

2020-10-05 14:36:51.002 | INFO     | pypads.app.base:activate_tracking:507 - Activating tracking...
2020-10-05 14:37:00.885 | INFO     | pypads.injections.setup.misc_setup:_call:72 - Tracking execution to run with id 3f37b1482453464b8cab677047b43c5d
2020-10-05 14:37:02.055 | WARNING  | pypads.app.misc.managed_git:preserve_changes:84 - There are uncommitted changes in your git!
2020-10-05 14:37:02.146 | INFO     | pypads.app.misc.managed_git:preserve_changes:96 - Using already existing pypads branch PyPads/f1ab4cea22ef4d45b03922e58fdb2d95
2020-10-05 14:37:02.361 | WARNING  | pypads.app.misc.managed_git:_handle_error:18 - Couldn't initialized git repository because of exception: Preserving commit failed due to INVALID_PARAMETER_VALUE: Tag value 'diff --git a/.idea/workspace.xml b/.idea/workspace.xml
new file mode 100644
index 0000000..08385ea
--- /dev/null
+++ b/.idea/workspace.xml
@@ -0,0 +1,229 @@
+<?xml version="1.0" encoding="UTF-8"?>
+<project version="4">
+  <component name="Branch

In [3]:
# Set the random seed to 1
tracker.actuators.set_random_seed(seed=1)

/home/christofer/PycharmProjects/pypads-notebooks/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/christofer/PycharmProjects/pypads-notebooks/venv/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:125: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


## Import the dataset from Scikit-learn

In [4]:
from sklearn import datasets

# load the iris datasets
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target


/home/christofer/PycharmProjects/pypads-notebooks/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Do the test train split

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

train = np.append(X_train, y_train.reshape((y_train.shape[0],1)), axis=1)
train.shape

/home/christofer/PycharmProjects/pypads-notebooks/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(90, 5)

## Create Torch class for accepting neural network shapes

In [6]:
import torch
from torch import nn
from typing import Any
from collections import OrderedDict

class Model(nn.Module):
    def _forward_unimplemented(self, *input: Any) -> None:
        pass
    
    def __init__(self, feature_shape, leaky_value=0.05, shape=[30, 50, 1]):
        super(Model, self).__init__()
        layers = [feature_shape] + shape
        layer_list = OrderedDict()
        for idx in range(1, len(layers)):
            layer_list[str(idx)] = nn.Linear(layers[idx-1], layers[idx])
            layer_list['Relu' + str(idx)] = nn.LeakyReLU(leaky_value, inplace=True)

        layer_list['Sigmoid'] = nn.Sigmoid()
        self.model = nn.Sequential(layer_list)

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity
    

In [7]:
# Define the various training parameters
num_features = X.shape[1]
num_labels = 3
num_epochs = 75
test_every = 10
h_lr = 0.005
h_b1 = 0.5
h_b2 = 0.999
batch_size = 10
neural_network_shape = [10, 10, 3]

In [8]:
# Create the model
model = Model(feature_shape=num_features, shape=neural_network_shape)

In [9]:
# Create the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=h_lr, betas=(h_b1, h_b2))

In [10]:
# Create the loss function
loss_ = torch.nn.BCELoss()

In [11]:
# Function to train for one epoch
from torch.utils.data import DataLoader

def train_one_epoch(model, loader: DataLoader, tensor, optimizer, loss_, scheduler=None):
    total_loss = 0
    n_iter = 0
    for batch_idx, (batch_train_features, batch_train_labels) in enumerate(loader):

        features_batch = tensor(batch_train_features.type(torch.FloatTensor))
        labels_batch = tensor(batch_train_labels)
        
        optimizer.zero_grad()

        # Compute loss
        forward = model.forward(features_batch)
        _loss = loss_(forward, labels_batch)

        _loss.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()

        total_loss += _loss
        n_iter += 1

    return total_loss / n_iter

In [12]:
# Function to split the batch into features and labels
def sample_collate(batch):
    one_hot = True
    num_labels = 3
    X = torch.stack([torch.tensor(x[:-1]) for x in batch])
    if one_hot is False:
        Y = [y[-1] for y in batch]
    else:
        Y = torch.zeros(len(batch), num_labels)
        for idx in range(len(batch)):
            Y[idx][int(batch[idx][-1])] = 1
            
    return X, Y

In [13]:
# Create the data loader for training
loader = DataLoader(train, batch_size=10, shuffle=True, collate_fn=sample_collate)

In [14]:
# Train the model

from time import time

labels = [i for i in range(num_labels)]

cuda = True if torch.cuda.is_available() else False
tensor = torch.cuda.FloatStorage if cuda else torch.FloatTensor

results = model.forward(tensor(X_test))
predicted = torch.max(results.data, 1)
y_pred = predicted.indices.tolist()
print('Initial accuracy is %f ' % accuracy_score(y_true=y_test.tolist(), y_pred=y_pred))

epochs_trained = 0

for idx in range(num_epochs):

    t1 = time()
    n_iter = 0
    total_loss = 0.0

    total_loss = train_one_epoch(model=model, loader=loader, optimizer=optimizer, loss_=loss_, tensor=tensor)

    t2 = time()
    print('Average loss during epoch %d: %f with execution time: %0.2f' % (idx, total_loss, (t2 - t1)))
    
    epochs_trained += 1
    if epochs_trained % test_every == 0:
        results = model.forward(tensor(X_test))
        predicted = torch.max(results.data, 1)
        y_pred = predicted.indices.tolist()
        print('Accuracy at %d epoch is %f ' % (epochs_trained, accuracy_score(y_true=y_test.tolist(), y_pred=y_pred)))

Initial accuracy is 0.633333 
Average loss during epoch 0: 0.682202 with execution time: 0.01
Average loss during epoch 1: 0.675866 with execution time: 0.01
Average loss during epoch 2: 0.668212 with execution time: 0.01
Average loss during epoch 3: 0.659434 with execution time: 0.01
Average loss during epoch 4: 0.648030 with execution time: 0.01
Average loss during epoch 5: 0.633524 with execution time: 0.01
Average loss during epoch 6: 0.615018 with execution time: 0.01
Average loss during epoch 7: 0.592594 with execution time: 0.01
Average loss during epoch 8: 0.537318 with execution time: 0.01
Average loss during epoch 9: 0.516329 with execution time: 0.01
Accuracy at 10 epoch is 0.650000 
Average loss during epoch 10: 0.505683 with execution time: 0.01
Average loss during epoch 11: 0.483489 with execution time: 0.01
Average loss during epoch 12: 0.473819 with execution time: 0.01
Average loss during epoch 13: 0.462664 with execution time: 0.01
Average loss during epoch 14: 0.4479

In [15]:
# Print the classification report
results = model.forward(tensor(X_test))
predicted = torch.max(results.data, 1)
y_pred = predicted.indices.tolist()
print(classification_report(y_true=y_test.tolist(), y_pred=y_pred, labels=labels))

2020-10-05 14:37:15.890 | WARNING  | pypads.injections.analysis.parameters:__post__:85 - No parameters are defined on the mapping file for <class 'sklearn.preprocessing._label.LabelEncoder'>. Trying to extract by other means...
2020-10-05 14:37:16.626 | WARNING  | pypads.injections.analysis.parameters:__post__:85 - No parameters are defined on the mapping file for <class 'sklearn.preprocessing._label.LabelEncoder'>. Trying to extract by other means...
2020-10-05 14:37:17.358 | WARNING  | pypads.injections.analysis.parameters:__post__:85 - No parameters are defined on the mapping file for <class 'sklearn.preprocessing._label.LabelEncoder'>. Trying to extract by other means...
2020-10-05 14:37:18.094 | WARNING  | pypads.injections.analysis.parameters:__post__:85 - No parameters are defined on the mapping file for <class 'sklearn.preprocessing._label.LabelEncoder'>. Trying to extract by other means...
              precision    recall  f1-score   support

           0       1.00      1.00

## Tracked Information

In [18]:
tracker.api.active_run().info

<RunInfo: artifact_uri='s3://pypads/18/5cca87cdda4d4193880c75ce3363f279/artifacts', end_time=None, experiment_id='18', lifecycle_stage='active', run_id='5cca87cdda4d4193880c75ce3363f279', run_uuid='5cca87cdda4d4193880c75ce3363f279', start_time=1601901422799, status='RUNNING', user_id='christofer'>

In [19]:
metrics = tracker.api.get_metrics(run_id=tracker.api.active_run().info.run_id, history=True)
print(metrics)

[]


### Get all the artefacts

In [23]:
artifacts = tracker.api.get_artifacts(run_id=tracker.api.active_run().info.run_id, path="*")
print(artifacts)

[ArtifactInfo(meta=ArtifactMetaModel(experiment_id='18', experiment_name='PyTorch Example', run_id='5cca87cdda4d4193880c75ce3363f279', created_at=1601901432.4173603, clazz="<class 'pypads.model.storage.ArtifactMetaModel'>", category='Artifact', uid=UUID('39c22a65-2773-4e59-a3c5-54980fc6c095'), description='', additional_data=None, path='Dependencies Run Setup Logger/Call/9c87d851-03b0-4ba9-870c-d82bcf74a0a9.json', file_format=<FileFormats.json: 'json'>, type='Call'), file_size=456), ArtifactInfo(meta=ArtifactMetaModel(experiment_id='18', experiment_name='PyTorch Example', run_id='5cca87cdda4d4193880c75ce3363f279', created_at=1601901431.7438211, clazz="<class 'pypads.model.storage.ArtifactMetaModel'>", category='Artifact', uid=UUID('6c9af162-1678-43ed-a443-7e99cc57d327'), description='dependency list from pip freeze', additional_data=None, path='Dependencies Run Setup Logger/GenericTrackedObject/76b83971-4ba5-4de4-9e30-53ba92d625c8/pip_freeze.txt', file_format=<FileFormats.text: 'txt'>,

### Metrics Information

In [24]:
metrics = tracker.api.get_metrics(run_id=tracker.api.active_run().info.run_id, history=True)
print(metrics)

[]


### Tracked Parameter Information

In [26]:
parameters = tracker.api.get_parameters(run_id=tracker.api.active_run().info.run_id)
print(parameters)

[]


### End tracking of experiment

In [28]:
tracker.api.end_run()